# ENSF-612 Quiz 1

## Preparation

I created a dummy file, and uploaded it at `dbfs:/FileStore/quiz_1/data.csv`. I load this file into pyspark dataframe.

*Citations - I have adapted the data for the dummy file from [here](http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Musical_Instruments_5.json.gz).*

In [0]:
def read_CSV_to_DF(filepath):
  """
  Read a csv file into a spark dataframe
  """
  df = (spark.read
        .option("multiline", "true")
        .option("quote", '"')
        .option("header", "true")
        .option("escape", "\\")
        .option("escape", '"')
        .csv(filepath)
        )
  
  return df

# creating the dataframe
df = read_CSV_to_DF('/FileStore/quiz_1/data.csv')

# updating the datatype of columns of dataframe
df = df.withColumn('NumberOfProductBought', df['NumberOfProductBought'].cast('int'))
df = df.withColumn('ReviewRating', df['ReviewRating'].cast('int'))

I created the below functions that emulate the functionality that we have to assume for this quiz.

In [0]:
import random
from datetime import datetime


@udf
def getSentiment(ReviewText):
  """
  returns 'p' for positive, 'n' for negative and 'o'
  for neutral polarity found in the ReviewText
  """
  
  sentiment = ['n', 'o', 'p']
  random.seed(len(ReviewText))
  return random.choice(sentiment)
      

@udf
def getUserCountry(UserId):
  """
  returns country of UserId
  """
  
  userCountry = ['Israel' , 'United States' , 'Germany' , 'Canada' , 'Denmark']
  random.seed(UserId)
  return random.choice(userCountry)


@udf
def getUserCity(UserId):
  """
  returns city of country if UserId
  """
  
  userCity = ['Tel Aviv', 'Jaffa', 'New York', 'Chicago', 'Munich', 'Berlin', 'Vancouver', 'Calgary', 'Copenhagen', 'Odense']
  random.seed(UserId)
  return random.choice(userCity)


@udf
def getYear(ReviewTime):
  """
  returns year of the ReviewTime
  """
  
  ReviewTime = int(ReviewTime)
  reviewYear = datetime.fromtimestamp(ReviewTime).strftime('%Y')
  return reviewYear


@udf
def getMonth(ReviewTime):
  """
  returns month of the ReviewTime
  """
  
  ReviewTime = int(ReviewTime)
  reviewMonth = datetime.fromtimestamp(ReviewTime).strftime('%m')
  return reviewMonth


@udf
def getDay(ReviewTime):
  """
  returns day of a week like Monday, Tuesday, Sunday
  """
  
  ReviewTime = int(ReviewTime)
  reviewDay = datetime.fromtimestamp(ReviewTime).strftime('%A')
  return reviewDay

## Task 1

In [0]:
# adding additional columns
df = df.select("*", getSentiment("ReviewText").alias("SentimentPolarity"))
df = df.select("*", getUserCountry("UserId").alias("UserCountry"))
df = df.select("*", getUserCity("UserId").alias("UserCity"))
df = df.select("*", getYear("ReviewTime").alias("ReviewYear"))
df = df.select("*", getMonth("ReviewTime").alias("ReviewMonth"))
df = df.select("*", getDay("ReviewTime").alias("ReviewDay"))

# showing the results
df.toPandas().head()   # df.show(n=100)

Out[3]:

,UserId,ReviewText,ReviewTime,ReviewRating,NumberOfProductBought,SentimentPolarity,UserCountry,UserCity,ReviewYear,ReviewMonth,ReviewDay
0,452526,"Not much to write about here, but it does exac...",1393545600,5,7,p,Germany,Berlin,2014,02,Friday
1,412630,The product does exactly as it should and is q...,1363392000,5,7,p,Canada,Vancouver,2013,03,Saturday
2,169420,The primary job of this device is to block the...,1377648000,5,1,o,Germany,Munich,2013,08,Wednesday
3,470797,Nice windscreen protects my MXL mic and preven...,1392336000,5,6,n,United States,New York,2014,02,Friday
4,412630,This pop filter is great. It looks and perform...,1392940800,5,2,o,Canada,Vancouver,2014,02,Friday


## Task 2

In [0]:
# total number of products
numberOfProducts = df.select('NumberOfProductBought').rdd.flatMap(lambda x: x).reduce(lambda x, y: x + y)

# printing the results
print("Total number of products = {}".format(numberOfProducts))

Total number of products = 293

## Task 3

#### Subtask 1

In [0]:
# sum all the ratings
ratingSum = df.select('ReviewRating').rdd.flatMap(lambda x: x).reduce(lambda x, y: x+y)

# divide by total to get average
ratingAverage = ratingSum/df.select('ReviewRating').count()

# printing the average rating by all users
print("Average rating = {}".format(ratingAverage))

Average rating = 4.52

#### Subtask 1 - alternate

In [0]:
# sum all the ratings and number of ratings by each user
rdd_sumRating = df.select(['UserId', 'ReviewRating']).rdd.map(lambda x: (x['UserId'], (x['ReviewRating'], 1))).reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))

# get the average of ratings
rdd_avgRating = rdd_sumRating.map(lambda x: (x[0], x[1][0]/x[1][1]))

# pretty print into table using df.show()
spark.createDataFrame(rdd_avgRating, ['UserID', 'Average Rating']).show(n=100)

+------+-----------------+
UserID| Average Rating|
+------+-----------------+
452526| 4.0|
412630| 4.4|
169420| 4.5|
470797| 5.0|
401219| 5.0|
129792| 4.0|
188774| 5.0|
200597|4.666666666666667|
317272| 5.0|
440556| 2.0|
106103| 4.8|
332956| 5.0|
420864| 4.0|
220114| 5.0|
327906| 5.0|
371303| 4.0|
443016| 5.0|
104350| 4.0|
276172| 5.0|
484954| 5.0|
203612| 4.0|
382701| 5.0|
297835| 5.0|
400624| 4.0|
491273| 4.0|
+------+-----------------+

#### Subtask 2a

In [0]:
# total number of reviews by polarity
rdd_reviewPolarity = df.select('SentimentPolarity').rdd.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a+b)
reviewsByPolarity = rdd_reviewPolarity.collect()

# printing the results
print("Number of positive reviews = {}\nNumber of neutral reviews  = {}\nNumber of negative Reviews = {}".format(reviewsByPolarity[0][1], reviewsByPolarity[1][1], reviewsByPolarity[2][1]))

Number of positive reviews = 10
Number of neutral reviews = 23
Number of negative Reviews = 17

#### Subtask 2b

In [0]:
# total number of reviews by country
rdd_reviewsByCountry = df.select(['UserCountry']).rdd.map(lambda x: (x['UserCountry'], 1)).reduceByKey(lambda a, b: a+b).sortByKey()

# pretty print into table using df.show()
spark.createDataFrame(rdd_reviewsByCountry, ['Country', 'Total Reviews']).show(n=100)

+-------------+-------------+
 Country|Total Reviews|
+-------------+-------------+
 Canada| 12|
 Denmark| 8|
 Germany| 12|
 Israel| 7|
United States| 11|
+-------------+-------------+

In [0]:
# total number of reviews by user
rdd_reviewsByUser = df.select(['UserId']).rdd.map(lambda x: (x['UserId'], 1)).reduceByKey(lambda a, b: a+b).sortByKey()

# pretty print into table using df.show()
spark.createDataFrame(rdd_reviewsByUser, ['UserId', 'Total Reviews']).show(n=100)

+------+-------------+
UserId|Total Reviews|
+------+-------------+
104350| 1|
106103| 5|
129792| 6|
169420| 2|
188774| 2|
200597| 3|
203612| 1|
220114| 1|
276172| 2|
297835| 1|
317272| 1|
327906| 1|
332956| 2|
371303| 1|
382701| 4|
400624| 1|
401219| 1|
412630| 5|
420864| 1|
440556| 1|
443016| 2|
452526| 3|
470797| 1|
484954| 1|
491273| 1|
+------+-------------+

#### Subtask 2b - alternate

In [0]:
# total number of reviews by country and by user
rdd = df.select(['UserCountry', 'UserId']).rdd
reviewsByUserCountry = rdd.map(lambda x: [(x['UserCountry'], x['UserId']), 1]).reduceByKey(lambda a, b: a+b).sortByKey()

# pretty print into table using custom code
iter = reviewsByUserCountry.toLocalIterator()
print("+---------------+---------+----------------+")
print("| {:>13} | {:>7} | {:>14} |".format("Country", "UserId", "Total Reviews"))
print("+---------------+---------+----------------+")
for row in iter:
  print("| {:>13} | {:>7} | {:>14} |".format(row[0][0], row[0][1], row[1]))
print("+---------------+---------+----------------+")

+---------------+---------+----------------+
 Country | UserId | Total Reviews |
+---------------+---------+----------------+
 Canada | 106103 | 5 |
 Canada | 220114 | 1 |
 Canada | 412630 | 5 |
 Canada | 420864 | 1 |
 Denmark | 104350 | 1 |
 Denmark | 129792 | 6 |
 Denmark | 440556 | 1 |
 Germany | 169420 | 2 |
 Germany | 276172 | 2 |
 Germany | 332956 | 2 |
 Germany | 443016 | 2 |
 Germany | 452526 | 3 |
 Germany | 491273 | 1 |
 Israel | 203612 | 1 |
 Israel | 371303 | 1 |
 Israel | 382701 | 4 |
 Israel | 400624 | 1 |
 United States | 188774 | 2 |
 United States | 200597 | 3 |
 United States | 297835 | 1 |
 United States | 317272 | 1 |
 United States | 327906 | 1 |
 United States | 401219 | 1 |
 United States | 470797 | 1 |
 United States | 484954 | 1 |
+---------------+---------+----------------+